# Import Statements and Initialisation

In [2]:
%run "..\Python Client Connector\python_client_connector".py
%load_ext autoreload
%autoreload
%matplotlib notebook
#Create connector instance, with particular port for PUSH, PULL and SUB
connector = MTConnector(VERBOSE=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
[INIT] Ready to send commands to METATRADER (PUSH): 32768
[INIT] Listening for responses from METATRADER (PULL): 32769
[INIT] Listening for market data from METATRADER (SUB): 32770


# Subscribing to Tick Data

In [3]:
connector.subscribeToSymbolTicks()
#connector.remoteReceive(connector.SUB_SOCKET)

[KERNEL] Subscribed to { BID/ASK updates. See self.MARKET_DATA_DB.
{'GBPUSD': {'2019.06.05 20:54:11:217': [1.26931, 1.26934, 0.0, 0.0, 4], '2019.06.05 20:54:11:808': [1.26932, 1.26934, 0.0, 0.0, 2]}}
GBPUSD
{'GBPUSD': {'2019.06.05 20:54:12:335': [1.26932, 1.26935, 0.0, 0.0, 4], '2019.06.05 20:54:12:508': [1.26932, 1.26936, 0.0, 0.0, 4], '2019.06.05 20:54:12:717': [1.26938, 1.26939, 0.0, 0.0, 6], '2019.06.05 20:54:12:722': [1.26938, 1.26941, 0.0, 0.0, 4], '2019.06.05 20:54:12:729': [1.26938, 1.26942, 0.0, 0.0, 4], '2019.06.05 20:54:12:736': [1.26938, 1.26943, 0.0, 0.0, 4], '2019.06.05 20:54:12:762': [1.26938, 1.26943, 0.0, 0.0, 4], '2019.06.05 20:54:12:768': [1.2694, 1.26943, 0.0, 0.0, 2], '2019.06.05 20:54:12:795': [1.26941, 1.26943, 0.0, 0.0, 2], '2019.06.05 20:54:12:829': [1.26941, 1.26944, 0.0, 0.0, 4], '2019.06.05 20:54:12:982': [1.26942, 1.26944, 0.0, 0.0, 2]}}
GBPUSD
{'GBPUSD': {'2019.06.05 20:54:13:350': [1.26942, 1.26945, 0.0, 0.0, 4]}}
GBPUSD
{'GBPUSD': {'2019.06.05 20:54:14:4

In [6]:
#connector.MARKET_DATA_DB['GBPUSD'].drop_duplicates(inplace=True)
test = connector.MARKET_DATA_DB['GBPUSD']
test

,Bid,Ask,Last,Vol,Flag
2019.06.05 20:54:11:217,1.26931,1.26934,0.0,0.0,4
2019.06.05 20:54:11:808,1.26932,1.26934,0.0,0.0,2
2019.06.05 20:54:12:335,1.26932,1.26935,0.0,0.0,4
2019.06.05 20:54:12:508,1.26932,1.26936,0.0,0.0,4
2019.06.05 20:54:12:717,1.26938,1.26939,0.0,0.0,6
2019.06.05 20:54:12:722,1.26938,1.26941,0.0,0.0,4
2019.06.05 20:54:12:729,1.26938,1.26942,0.0,0.0,4
2019.06.05 20:54:12:736,1.26938,1.26943,0.0,0.0,4
2019.06.05 20:54:12:762,1.26938,1.26943,0.0,0.0,4
2019.06.05 20:54:12:768,1.26940,1.26943,0.0,0.0,2


## import pandas as pd


In [13]:
import pandas as pd
test.index = pd.to_datetime(test.index,format="%Y.%m.%d %H:%M:%S:%f")
test = test.rename_axis('Time')
test

{'GBPUSD': {'2019.06.06 02:52:03:651': [1.26885, 1.26892, 0.0, 0.0, 2]}}
GBPUSD


,Bid,Ask,Last,Vol,Flag
Time,,,,,
2019-06-05 20:54:11.217,1.26931,1.26934,0.0,0.0,4
2019-06-05 20:54:11.808,1.26932,1.26934,0.0,0.0,2
2019-06-05 20:54:12.335,1.26932,1.26935,0.0,0.0,4
2019-06-05 20:54:12.508,1.26932,1.26936,0.0,0.0,4
2019-06-05 20:54:12.717,1.26938,1.26939,0.0,0.0,6
2019-06-05 20:54:12.722,1.26938,1.26941,0.0,0.0,4
2019-06-05 20:54:12.729,1.26938,1.26942,0.0,0.0,4
2019-06-05 20:54:12.736,1.26938,1.26943,0.0,0.0,4
2019-06-05 20:54:12.762,1.26938,1.26943,0.0,0.0,4


{'GBPUSD': {'2019.06.06 02:52:04:71': [1.26886, 1.26892, 0.0, 0.0, 2]}}
GBPUSD
{'GBPUSD': {'2019.06.06 02:52:06:167': [1.26885, 1.26892, 0.0, 0.0, 2], '2019.06.06 02:52:06:194': [1.26886, 1.26892, 0.0, 0.0, 2]}}
GBPUSD
{'GBPUSD': {'2019.06.06 02:52:11:202': [1.26884, 1.26891, 0.0, 0.0, 6], '2019.06.06 02:52:11:209': [1.26884, 1.26889, 0.0, 0.0, 4], '2019.06.06 02:52:11:261': [1.26882, 1.26889, 0.0, 0.0, 2], '2019.06.06 02:52:11:275': [1.26881, 1.26889, 0.0, 0.0, 2], '2019.06.06 02:52:11:282': [1.26881, 1.26888, 0.0, 0.0, 4], '2019.06.06 02:52:11:419': [1.26881, 1.26887, 0.0, 0.0, 4], '2019.06.06 02:52:11:472': [1.2688, 1.26887, 0.0, 0.0, 2]}}
GBPUSD
{'GBPUSD': {'2019.06.06 02:52:16:562': [1.2688, 1.26886, 0.0, 0.0, 4], '2019.06.06 02:52:16:602': [1.26879, 1.26886, 0.0, 0.0, 2], '2019.06.06 02:52:16:615': [1.26879, 1.26885, 0.0, 0.0, 4], '2019.06.06 02:52:16:629': [1.26878, 1.26885, 0.0, 0.0, 2], '2019.06.06 02:52:16:697': [1.26877, 1.26885, 0.0, 0.0, 2], '2019.06.06 02:52:16:739': [1.2

In [ ]:
# tickdataframe = pd.DataFrame(connector.MARKET_DATA_DB['GBPUSD'])

In [ ]:
# tickdataframe = tickdataframe[['Time','Bid','Ask','Flag']]
# tickdataframe

In [14]:
from pandas import HDFStore
hdf = HDFStore('testTickData.h5')

In [15]:
hdf.put('testdata',test,format='table',data_columns=True)

In [16]:
hdf['testdata']

,Bid,Ask,Last,Vol,Flag
Time,,,,,
2019-06-05 20:54:11.217,1.26931,1.26934,0.0,0.0,4
2019-06-05 20:54:11.808,1.26932,1.26934,0.0,0.0,2
2019-06-05 20:54:12.335,1.26932,1.26935,0.0,0.0,4
2019-06-05 20:54:12.508,1.26932,1.26936,0.0,0.0,4
2019-06-05 20:54:12.717,1.26938,1.26939,0.0,0.0,6
2019-06-05 20:54:12.722,1.26938,1.26941,0.0,0.0,4
2019-06-05 20:54:12.729,1.26938,1.26942,0.0,0.0,4
2019-06-05 20:54:12.736,1.26938,1.26943,0.0,0.0,4
2019-06-05 20:54:12.762,1.26938,1.26943,0.0,0.0,4


{'GBPUSD': {'2019.06.06 02:52:36:574': [1.26877, 1.26884, 0.0, 0.0, 4], '2019.06.06 02:52:36:587': [1.26878, 1.26884, 0.0, 0.0, 2], '2019.06.06 02:52:36:602': [1.26877, 1.26884, 0.0, 0.0, 2], '2019.06.06 02:52:36:635': [1.26876, 1.26884, 0.0, 0.0, 2], '2019.06.06 02:52:36:702': [1.26877, 1.26884, 0.0, 0.0, 2], '2019.06.06 02:52:36:758': [1.26876, 1.26884, 0.0, 0.0, 2], '2019.06.06 02:52:36:784': [1.26877, 1.26884, 0.0, 0.0, 2], '2019.06.06 02:52:36:884': [1.26878, 1.26884, 0.0, 0.0, 2], '2019.06.06 02:52:36:891': [1.26879, 1.26886, 0.0, 0.0, 4], '2019.06.06 02:52:36:892': [1.26879, 1.26887, 0.0, 0.0, 4], '2019.06.06 02:52:36:898': [1.2688, 1.26888, 0.0, 0.0, 2], '2019.06.06 02:52:36:904': [1.2688, 1.26887, 0.0, 0.0, 4], '2019.06.06 02:52:36:930': [1.2688, 1.26888, 0.0, 0.0, 4], '2019.06.06 02:52:36:944': [1.26881, 1.26888, 0.0, 0.0, 2], '2019.06.06 02:52:36:957': [1.26883, 1.26888, 0.0, 0.0, 2], '2019.06.06 02:52:36:964': [1.26884, 1.2689, 0.0, 0.0, 2], '2019.06.06 02:52:36:971': [1.26

{'GBPUSD': {'2019.06.06 02:54:13:231': [1.26878, 1.26886, 0.0, 0.0, 2]}}
GBPUSD
{'GBPUSD': {'2019.06.06 02:54:14:161': [1.26878, 1.26885, 0.0, 0.0, 4]}}
GBPUSD
{'GBPUSD': {'2019.06.06 02:54:16:452': [1.26881, 1.26885, 0.0, 0.0, 2], '2019.06.06 02:54:16:953': [1.26878, 1.26885, 0.0, 0.0, 2]}}
GBPUSD
{'GBPUSD': {'2019.06.06 02:54:17:843': [1.26878, 1.26884, 0.0, 0.0, 4]}}
GBPUSD
{'GBPUSD': {'2019.06.06 02:54:18:257': [1.26876, 1.26884, 0.0, 0.0, 2], '2019.06.06 02:54:18:505': [1.26877, 1.26884, 0.0, 0.0, 2]}}
GBPUSD
{'GBPUSD': {'2019.06.06 02:54:19:638': [1.26876, 1.26884, 0.0, 0.0, 2]}}
GBPUSD
{'GBPUSD': {'2019.06.06 02:54:30:881': [1.26876, 1.26885, 0.0, 0.0, 4]}}
GBPUSD
{'GBPUSD': {'2019.06.06 02:54:36:300': [1.26875, 1.26884, 0.0, 0.0, 6], '2019.06.06 02:54:36:378': [1.26875, 1.26883, 0.0, 0.0, 4], '2019.06.06 02:54:36:392': [1.26875, 1.26884, 0.0, 0.0, 4], '2019.06.06 02:54:36:475': [1.26876, 1.26884, 0.0, 0.0, 2], '2019.06.06 02:54:36:582': [1.26875, 1.26884, 0.0, 0.0, 2]}}
GBPUSD


{'GBPUSD': {'2019.06.06 02:58:43:182': [1.26876, 1.26885, 0.0, 0.0, 4]}}
GBPUSD
{'GBPUSD': {'2019.06.06 02:58:59:469': [1.26876, 1.26884, 0.0, 0.0, 4]}}
GBPUSD
{'GBPUSD': {'2019.06.06 02:59:07:406': [1.26876, 1.26884, 0.0, 0.0, 2]}}
GBPUSD
{'GBPUSD': {'2019.06.06 02:59:12:761': [1.26876, 1.26883, 0.0, 0.0, 4], '2019.06.06 02:59:12:774': [1.26875, 1.26883, 0.0, 0.0, 2], '2019.06.06 02:59:12:828': [1.26876, 1.26884, 0.0, 0.0, 2]}}
GBPUSD
{'GBPUSD': {'2019.06.06 02:59:13:148': [1.26875, 1.26884, 0.0, 0.0, 2], '2019.06.06 02:59:13:433': [1.26876, 1.26884, 0.0, 0.0, 2]}}
GBPUSD
{'GBPUSD': {'2019.06.06 02:59:18:538': [1.26876, 1.26885, 0.0, 0.0, 4]}}
GBPUSD
{'GBPUSD': {'2019.06.06 02:59:25:265': [1.26876, 1.26884, 0.0, 0.0, 4]}}
GBPUSD
{'GBPUSD': {'2019.06.06 02:59:32:176': [1.26875, 1.26884, 0.0, 0.0, 2], '2019.06.06 02:59:32:184': [1.26875, 1.26884, 0.0, 0.0, 4], '2019.06.06 02:59:32:191': [1.26874, 1.26884, 0.0, 0.0, 2], '2019.06.06 02:59:32:430': [1.26875, 1.26884, 0.0, 0.0, 2], '2019.06

{'GBPUSD': {'2019.06.06 03:00:24:136': [1.26867, 1.26874, 0.0, 0.0, 6], '2019.06.06 03:00:24:157': [1.26868, 1.26874, 0.0, 0.0, 2], '2019.06.06 03:00:24:210': [1.26869, 1.26875, 0.0, 0.0, 6]}}
GBPUSD
{'GBPUSD': {'2019.06.06 03:00:26:511': [1.26868, 1.26875, 0.0, 0.0, 2]}}
GBPUSD
{'GBPUSD': {'2019.06.06 03:00:28:786': [1.26868, 1.26876, 0.0, 0.0, 4]}}
GBPUSD
{'GBPUSD': {'2019.06.06 03:00:31:328': [1.26869, 1.26876, 0.0, 0.0, 2]}}
GBPUSD
{'GBPUSD': {'2019.06.06 03:00:32:136': [1.26868, 1.26876, 0.0, 0.0, 2], '2019.06.06 03:00:32:170': [1.26868, 1.26875, 0.0, 0.0, 4], '2019.06.06 03:00:32:758': [1.26867, 1.26875, 0.0, 0.0, 2], '2019.06.06 03:00:32:763': [1.26866, 1.26874, 0.0, 0.0, 4], '2019.06.06 03:00:32:771': [1.26866, 1.26873, 0.0, 0.0, 4], '2019.06.06 03:00:32:791': [1.26867, 1.26874, 0.0, 0.0, 6], '2019.06.06 03:00:32:932': [1.26867, 1.26875, 0.0, 0.0, 4], '2019.06.06 03:00:32:945': [1.26866, 1.26875, 0.0, 0.0, 2], '2019.06.06 03:00:32:969': [1.26866, 1.26874, 0.0, 0.0, 4], '2019.06

...

{'GBPUSD': {'2019.06.06 03:23:58:853': [1.26897, 1.26906, 0.0, 0.0, 2]}}
GBPUSD
{'GBPUSD': {'2019.06.06 03:24:00:153': [1.26898, 1.26906, 0.0, 0.0, 2]}}
GBPUSD
{'GBPUSD': {'2019.06.06 03:24:11:113': [1.26898, 1.26907, 0.0, 0.0, 4], '2019.06.06 03:24:11:152': [1.26898, 1.26906, 0.0, 0.0, 4], '2019.06.06 03:24:11:159': [1.26898, 1.26907, 0.0, 0.0, 4], '2019.06.06 03:24:11:180': [1.26899, 1.26907, 0.0, 0.0, 2], '2019.06.06 03:24:11:187': [1.26899, 1.26906, 0.0, 0.0, 4], '2019.06.06 03:24:11:200': [1.26899, 1.26907, 0.0, 0.0, 4], '2019.06.06 03:24:11:260': [1.269, 1.26907, 0.0, 0.0, 2], '2019.06.06 03:24:11:287': [1.269, 1.26907, 0.0, 0.0, 4]}}
GBPUSD
{'GBPUSD': {'2019.06.06 03:24:12:943': [1.26899, 1.26907, 0.0, 0.0, 2], '2019.06.06 03:24:12:951': [1.26899, 1.26906, 0.0, 0.0, 4], '2019.06.06 03:24:12:958': [1.26898, 1.26906, 0.0, 0.0, 2]}}
GBPUSD
{'GBPUSD': {'2019.06.06 03:24:13:14': [1.26897, 1.26906, 0.0, 0.0, 2]}}
GBPUSD
{'GBPUSD': {'2019.06.06 03:24:14:745': [1.26898, 1.26906, 0.0, 0.

{'GBPUSD': {'2019.06.06 03:28:25:169': [1.26884, 1.26892, 0.0, 0.0, 2], '2019.06.06 03:28:25:249': [1.26885, 1.26892, 0.0, 0.0, 2], '2019.06.06 03:28:25:256': [1.26884, 1.26892, 0.0, 0.0, 2], '2019.06.06 03:28:25:282': [1.26884, 1.26891, 0.0, 0.0, 4], '2019.06.06 03:28:25:349': [1.26885, 1.26892, 0.0, 0.0, 2], '2019.06.06 03:28:25:404': [1.26886, 1.26892, 0.0, 0.0, 2], '2019.06.06 03:28:25:463': [1.26886, 1.26894, 0.0, 0.0, 4], '2019.06.06 03:28:25:889': [1.26887, 1.26893, 0.0, 0.0, 2], '2019.06.06 03:28:25:903': [1.26887, 1.26896, 0.0, 0.0, 4], '2019.06.06 03:28:25:909': [1.26888, 1.26896, 0.0, 0.0, 2]}}
GBPUSD
{'GBPUSD': {'2019.06.06 03:28:26:383': [1.26888, 1.26897, 0.0, 0.0, 4]}}
GBPUSD
{'GBPUSD': {'2019.06.06 03:28:27:487': [1.26888, 1.26896, 0.0, 0.0, 4], '2019.06.06 03:28:27:896': [1.26888, 1.26897, 0.0, 0.0, 4]}}
GBPUSD
{'GBPUSD': {'2019.06.06 03:28:29:761': [1.26889, 1.26897, 0.0, 0.0, 2], '2019.06.06 03:28:29:780': [1.26889, 1.26896, 0.0, 0.0, 4], '2019.06.06 03:28:29:787': [

{'GBPUSD': {'2019.06.06 03:30:22:394': [1.26887, 1.26893, 0.0, 0.0, 2]}}
GBPUSD
{'GBPUSD': {'2019.06.06 03:30:23:683': [1.26888, 1.26893, 0.0, 0.0, 2], '2019.06.06 03:30:23:779': [1.26888, 1.26894, 0.0, 0.0, 4]}}
GBPUSD
{'GBPUSD': {'2019.06.06 03:30:33:582': [1.26888, 1.26895, 0.0, 0.0, 4], '2019.06.06 03:30:33:609': [1.26889, 1.26895, 0.0, 0.0, 2], '2019.06.06 03:30:33:641': [1.2689, 1.26897, 0.0, 0.0, 2]}}
GBPUSD
{'GBPUSD': {'2019.06.06 03:30:34:880': [1.26889, 1.26897, 0.0, 0.0, 2]}}
GBPUSD
{'GBPUSD': {'2019.06.06 03:30:43:912': [1.26888, 1.26897, 0.0, 0.0, 2]}}
GBPUSD
{'GBPUSD': {'2019.06.06 03:30:44:904': [1.26888, 1.26896, 0.0, 0.0, 4]}}
GBPUSD
{'GBPUSD': {'2019.06.06 03:30:50:682': [1.26888, 1.26895, 0.0, 0.0, 4]}}
GBPUSD
{'GBPUSD': {'2019.06.06 03:30:51:587': [1.26887, 1.26895, 0.0, 0.0, 2]}}
GBPUSD
{'GBPUSD': {'2019.06.06 03:31:00:100': [1.26887, 1.26895, 0.0, 0.0, 2], '2019.06.06 03:31:00:358': [1.26887, 1.26894, 0.0, 0.0, 4], '2019.06.06 03:31:00:425': [1.26887, 1.26893, 0.0

{'GBPUSD': {'2019.06.06 03:35:11:423': [1.26896, 1.26902, 0.0, 0.0, 2], '2019.06.06 03:35:11:435': [1.26897, 1.26902, 0.0, 0.0, 2]}}
GBPUSD
{'GBPUSD': {'2019.06.06 03:35:14:245': [1.26896, 1.26902, 0.0, 0.0, 2], '2019.06.06 03:35:14:482': [1.26897, 1.26902, 0.0, 0.0, 2]}}
GBPUSD
{'GBPUSD': {'2019.06.06 03:35:17:617': [1.26897, 1.26901, 0.0, 0.0, 4]}}
GBPUSD
{'GBPUSD': {'2019.06.06 03:35:19:495': [1.26897, 1.26902, 0.0, 0.0, 4]}}
GBPUSD
{'GBPUSD': {'2019.06.06 03:35:29:933': [1.26897, 1.26903, 0.0, 0.0, 4]}}
GBPUSD
{'GBPUSD': {'2019.06.06 03:35:31:829': [1.26897, 1.26901, 0.0, 0.0, 4], '2019.06.06 03:35:31:896': [1.26896, 1.26901, 0.0, 0.0, 2], '2019.06.06 03:35:31:904': [1.26895, 1.269, 0.0, 0.0, 2], '2019.06.06 03:35:31:918': [1.26892, 1.269, 0.0, 0.0, 2], '2019.06.06 03:35:31:938': [1.26892, 1.26898, 0.0, 0.0, 4], '2019.06.06 03:35:31:964': [1.26891, 1.26898, 0.0, 0.0, 2], '2019.06.06 03:35:31:989': [1.26891, 1.26899, 0.0, 0.0, 4]}}
GBPUSD
{'GBPUSD': {'2019.06.06 03:35:36:61': [1.268

{'GBPUSD': {'2019.06.06 03:38:09:21': [1.26898, 1.26905, 0.0, 0.0, 2], '2019.06.06 03:38:09:81': [1.26899, 1.26905, 0.0, 0.0, 2], '2019.06.06 03:38:09:142': [1.26898, 1.26905, 0.0, 0.0, 2], '2019.06.06 03:38:09:201': [1.26899, 1.26905, 0.0, 0.0, 2], '2019.06.06 03:38:09:260': [1.26898, 1.26905, 0.0, 0.0, 2], '2019.06.06 03:38:09:322': [1.26899, 1.26905, 0.0, 0.0, 2], '2019.06.06 03:38:09:379': [1.26898, 1.26905, 0.0, 0.0, 2], '2019.06.06 03:38:09:445': [1.26899, 1.26905, 0.0, 0.0, 2], '2019.06.06 03:38:09:560': [1.26898, 1.26905, 0.0, 0.0, 2], '2019.06.06 03:38:09:623': [1.26899, 1.26905, 0.0, 0.0, 2]}}
GBPUSD
{'GBPUSD': {'2019.06.06 03:38:11:432': [1.26898, 1.26905, 0.0, 0.0, 2], '2019.06.06 03:38:11:494': [1.26899, 1.26905, 0.0, 0.0, 2]}}
GBPUSD
{'GBPUSD': {'2019.06.06 03:38:13:99': [1.26898, 1.26905, 0.0, 0.0, 2], '2019.06.06 03:38:13:152': [1.26897, 1.26905, 0.0, 0.0, 2], '2019.06.06 03:38:13:158': [1.26897, 1.26906, 0.0, 0.0, 4], '2019.06.06 03:38:13:174': [1.26897, 1.26906, 0.0, 

# Loading data from HD5 File

In [ ]:
readDataFrame = pd.read_hdf('testTickData.h5')
readDataFrame.set_index('Time').loc[:,'Bid':'Ask']
#truncatedDataFrame = readDataFrame
#truncatedDataFrame['Time'] = truncatedDataFrame['Time'].dt.floor('S')
#truncatedDataFrame

# Calculate OHLC Data

In [ ]:
OHLCData = readDataFrame.resample('1min').agg({'Bid':'ohlc','Ask':'ohlc','Flag':'max'})
OHLCData

In [ ]:
OHLCData.loc[:,'Bid'].plot()